1. 准备 deepseek 客户端

In [1]:
import os
from openai import OpenAI

api_key = os.getenv("DEEPSEEK_API_KEY")

if not api_key:
    raise ValueError("请先设置 deepseek api key")

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1"
)

2. 准备商品向量数据库

In [28]:
!pip install "pymilvus[model]==2.5.10" openai==1.82.0 requests==2.32.3 tqdm==4.67.1 torch==2.7.0 sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: http://mirrors.cloud.aliyuncs.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 19.6 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [sentence-transformers]ence-transformers]


In [9]:
from glob import glob

text_lines = []

for file_path in glob("./rednote_product.md", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    text_lines += file_text.split("\n")

In [10]:
text_lines

['1. **冰川极地保湿精华**：通过-50℃活性保存技术，含极地雪绒花+玻尿酸+冰川水，为干性/敏感肌提供深层保湿，洁面后取3滴掌心搓热按压全脸。',
 '2. **黑松露抗老面霜**：运用诺贝尔细胞自噬技术，含黑松露发酵物+胜肽复合物，专为30+熟龄肌抗老设计，夜间厚敷按摩至吸收。',
 '3. **蓝铜胜肽修护面膜**：采用微电流导入式修护，含1%蓝铜胜肽+积雪草苷，针对医美术后肌屏障修复，每周敷15分钟免洗。',
 '4. **茶多酚控油洁颜粉**：实现吸油不拔干效果，含日本抹茶粉+海藻多糖，专为油痘肌清洁设计，加水揉泡后按摩T区30秒。',
 '5. **月光酵母精华水**：搭载夜间光修复技术，含清酒酵母+月光石微粒，改善暗沉肌肤色，化妆棉湿敷10分钟后轻拍。',
 '6. **钻石微晶眼膜**：创新3D剪裁抗引力设计，含六胜肽+钻石粉末，紧致眼周松弛肌肤，贴敷8分钟揭下无需清洗。',
 '7. **火山泥毛孔清洁膏**：采用磁吸式清洁技术，含济州岛火山泥+活性炭，深层净化毛孔粗大肌肤，每周厚敷10分钟温水洗净。',
 '8. **益生菌平衡乳液**：通过皮肤微生态调节技术，含10种益生菌+β-葡聚糖，平衡混合肌油脂分泌，早晚按压两泵全脸涂抹。',
 '9. **燕窝胶原唇膜**：运用3D水光膜技术，含水解胶原蛋白+燕窝提取物，修护干燥唇部，睡前厚涂次日擦拭。',
 '10. **龙血树修复精华**：激活植物干细胞再生，含龙血树脂+积雪草苷，快速修复受损肌肤，点涂创面每日2次。',
 '11. **海藻荧光面膜**：采用生物冷光焕活技术，含深海藻类荧光素+富勒烯，针对熬夜肌暗沉，薄涂20分钟洗去重现透亮。',
 '12. **蜂毒冻干安瓶**：运用微针渗透科技，含蜂毒肽+冻干VC粉，精准淡化法令纹，每晚1支点涂按摩至吸收。',
 '13. **陨石微粒磨砂膏**：创新行星级抛光系统，含陨石矿物微粒+木瓜蛋白酶，改善肌肤粗糙，湿手打圈按摩30秒冲净。',
 '14. **水母胶原喷雾**：搭载气溶胶锁水膜技术，含水母胶原蛋白+南极冰藻，即时缓解沙漠肌干燥，妆前妆后喷洒3秒成膜。',
 '15. **藏红花修护油**：运用古法淬炼工艺，含西藏藏红花+摩洛哥坚果油，修复孕纹，每日沐浴后顺时针按摩5分钟。',
 '16. **AI智能防晒贴**：具备紫外线感应变

In [29]:
# from pymilvus import model as milvus_model
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L12-v2")

In [31]:
# 创建数据库
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./my_milvus.db")

collection_name = "rednote_product"

In [32]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

In [33]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=384,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",
)

In [34]:
from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode(text_lines)

for i, line in enumerate(tqdm(text_lines, desc="Create Embedding")):
    data.append({ "id": i, "vector": doc_embeddings[i], "text": line })

milvus_client.insert(collection_name=collection_name, data=data)

Create Embedding: 100%|██████████| 20/20 [00:00<00:00, 38164.73it/s]


{'insert_count': 20, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], 'cost': 0}

In [2]:
# 设置提示词
SYSTEM_PROMPT = """
你是一个资深的小红书爆款文案专家，擅长结合最新潮流和产品卖点，创作引人入胜、高互动、高转化的笔记文案。

你的任务是根据用户提供的产品和需求，生成包含标题、正文、相关标签和表情符号的完整小红书笔记。

请始终采用'Thought-Action-Observation'模式进行推理和行动。文案风格需活泼、真诚、富有感染力。当完成任务后，请以JSON格式直接输出最终文案，格式如下：
```json
{
  "title": "小红书标题",
  "body": "小红书正文",
  "hashtags": ["#标签1", "#标签2", "#标签3", "#标签4", "#标签5"],
  "emojis": ["✨", "🔥", "💖"]
}
```
在生成文案前，请务必先思考并收集足够的信息。
"""

deepseek 工具定义规则
type: 目前只支持 function
function:
    name: 函数名
    description: 函数作用，并不仅仅是描述，简单理解就是需要让ai明白何时调用这个函数，所以需要当成函数的提示词来写，要写清楚函数的作用，可以适当举例用户要生成什么文案的时候调用
    parameters: 函数的参数定义，遵循 JSON Schema 规范
        type: 不定个数的函数参数应该是个对象，所以在这里固定使用 object
        properties: 对象的属性，type=object 固有配置
            query: 自定义的属性，可以为其他字段，只函数的形参定义
                type: 函数参数的类型，
                description: 函数参数的描述，同样需要提示词，用于从用户的提供的生成文案的提示词中提取相关的提示词进行查询
        required: 对象必须包含的属性

deepseek 本身并不会自动调用函数，而是返回需要调用的函数名和所需参数，由开发手动调用
    

In [3]:
# 工具定义
TOOLS_DEFINITION = [
    {
        "type": "function",
        "function": {
            "name": "search_web",
            "description": "搜索互联网上的实时信息，用于获取最新新闻、流行趋势、用户评价、行业报告等。请确保搜索关键词精确，避免宽泛的查询。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "要搜索的关键词或问题，例如'最新小红书美妆趋势'或'深海蓝藻保湿面膜 用户评价'"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "query_product_database",
            "description": "查询内部产品数据库，获取指定产品的详细卖点、成分、适用人群、使用方法等信息。",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_name": {
                        "type": "string",
                        "description": "要查询的产品名称，例如'深海蓝藻保湿面膜'"
                    }
                },
                "required": ["product_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "generate_emoji",
            "description": "根据提供的文本内容，生成一组适合小红书风格的表情符号。",
            "parameters": {
                "type": "object",
                "properties": {
                    "context": {
                        "type": "string",
                        "description": "文案的关键内容或情感，例如'惊喜效果'、'补水保湿'"
                    }
                },
                "required": ["context"]
            }
        }
    }
]

In [35]:
# 实现具体的函数
import random # 用于模拟生成表情
import time # 用于模拟网络延迟

# 模拟查询
def search_web(query: str) -> str:
    """模拟网页搜索工具，返回预设的搜索结果。"""
    print(f"[Tool Call] 模拟搜索网页：{query}")
    time.sleep(1) # 模拟网络延迟
    if "小红书美妆趋势" in query:
        return "近期小红书美妆流行'多巴胺穿搭'、'早C晚A'护肤理念、'伪素颜'妆容，热门关键词有#氛围感、#抗老、#屏障修复。"
    elif "保湿面膜" in query:
        return "小红书保湿面膜热门话题：沙漠干皮救星、熬夜急救面膜、水光肌养成。用户痛点：卡粉、泛红、紧绷感。"
    elif "深海蓝藻保湿面膜" in query:
        return "关于深海蓝藻保湿面膜的用户评价：普遍反馈补水效果好，吸收快，对敏感肌友好。有用户提到价格略高，但效果值得。"
    else:
        return f"未找到关于 '{query}' 的特定信息，但市场反馈通常关注产品成分、功效和用户体验。"

# 模拟查询产品数据库
def query_product_database(product_name: str) -> str:
    """模拟查询产品数据库，返回预设的产品信息。"""
    print(f"[Tool Call] 查询 milvus 产品数据库：{product_name}")
    search_res = milvus_client.search(
        collection_name=collection_name,
        data=embedding_model.encode([product_name]),
        limit=1,
        search_params={"metric_type": "IP", "params": {}},  # 内积距离
        output_fields=["text"],  # 返回 text 字段
    )
    return search_res[0][0].entity.text

# 模拟生成表情符号
def generate_emoji(context: str) -> list:
    """模拟生成表情符号，根据上下文提供常用表情。"""
    print(f"[Tool Call] 模拟生成表情符号，上下文：{context}")
    time.sleep(0.2) # 模拟生成延迟
    if "补水" in context or "水润" in context or "保湿" in context:
        return ["💦", "💧", "🌊", "✨"]
    elif "惊喜" in context or "哇塞" in context or "爱了" in context:
        return ["💖", "😍", "🤩", "💯"]
    elif "熬夜" in context or "疲惫" in context:
        return ["😭", "😮‍💨", "😴", "💡"]
    elif "好物" in context or "推荐" in context:
        return ["✅", "👍", "⭐", "🛍️"]
    else:
        return random.sample(["✨", "🔥", "💖", "💯", "🎉", "👍", "🤩", "💧", "🌿"], k=min(5, len(context.split())))

available_tools = {
    "search_web": search_web,
    "query_product_database": query_product_database,
    "generate_emoji": generate_emoji,
}

In [5]:
# agent 核心代码
import json
import re

def generate_rednote(product_name: str, tone_style: str = "活泼甜美", max_iterations: int = 5) -> str:
    """
    使用 DeepSeek Agent 生成小红书爆款文案。
    
    Args:
        product_name (str): 要生成文案的产品名称。
        tone_style (str): 文案的语气和风格，如"活泼甜美"、"知性"、"搞怪"等。
        max_iterations (int): Agent 最大迭代次数，防止无限循环。
        
    Returns:
        str: 生成的爆款文案（JSON 格式字符串）。
    """
    
    print(f"\n🚀 启动小红书文案生成助手，产品：{product_name}，风格：{tone_style}\n")
    
    # 存储对话历史，包括系统提示词和用户请求
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"请为产品「{product_name}」生成一篇小红书爆款文案。要求：语气{tone_style}，包含标题、正文、至少5个相关标签和5个表情符号。请以完整的JSON格式输出，并确保JSON内容用markdown代码块包裹（例如：```json{{...}}```）。"}
    ]
    
    iteration_count = 0
    final_response = None
    
    while iteration_count < max_iterations:
        iteration_count += 1
        print(f"-- Iteration {iteration_count} --")
        
        try:
            # 调用 DeepSeek API，传入对话历史和工具定义
            response = deepseek_client.chat.completions.create(
                model="deepseek-chat",
                messages=messages,
                tools=TOOLS_DEFINITION, # 告知模型可用的工具
                tool_choice="auto" # 允许模型自动决定是否使用工具
            )

            response_message = response.choices[0].message
            
            # **ReAct模式：处理工具调用**
            if response_message.tool_calls: # 如果模型决定调用工具
                print("Agent: 决定调用工具...")
                messages.append(response_message) # 将工具调用信息添加到对话历史
                
                tool_outputs = []
                for tool_call in response_message.tool_calls:
                    function_name = tool_call.function.name
                    # 确保参数是合法的JSON字符串，即使工具不要求参数，也需要传递空字典
                    function_args = json.loads(tool_call.function.arguments) if tool_call.function.arguments else {}

                    print(f"Agent Action: 调用工具 '{function_name}'，参数：{function_args}")
                    
                    # 查找并执行对应的模拟工具函数
                    if function_name in available_tools:
                        tool_function = available_tools[function_name]
                        tool_result = tool_function(**function_args)
                        print(f"Observation: 工具返回结果：{tool_result}")
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": str(tool_result) # 工具结果作为字符串返回
                        })
                    else:
                        error_message = f"错误：未知的工具 '{function_name}'"
                        print(error_message)
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": error_message
                        })
                messages.extend(tool_outputs) # 将工具执行结果作为 Observation 添加到对话历史
                
            # **ReAct 模式：处理最终内容**
            elif response_message.content: # 如果模型直接返回内容（通常是最终答案）
                print(f"[模型生成结果] {response_message.content}")
                
                # --- START: 添加 JSON 提取和解析逻辑 ---
                json_string_match = re.search(r"```json\s*(\{.*\})\s*```", response_message.content, re.DOTALL)
                
                if json_string_match:
                    extracted_json_content = json_string_match.group(1)
                    try:
                        final_response = json.loads(extracted_json_content)
                        print("Agent: 任务完成，成功解析最终JSON文案。")
                        return json.dumps(final_response, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError as e:
                        print(f"Agent: 提取到JSON块但解析失败: {e}")
                        print(f"尝试解析的字符串:\n{extracted_json_content}")
                        messages.append(response_message) # 解析失败，继续对话
                else:
                    # 如果没有匹配到 ```json 块，尝试直接解析整个 content
                    try:
                        final_response = json.loads(response_message.content)
                        print("Agent: 任务完成，直接解析最终JSON文案。")
                        return json.dumps(final_response, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError:
                        print("Agent: 生成了非JSON格式内容或非Markdown JSON块，可能还在思考或出错。")
                        messages.append(response_message) # 非JSON格式，继续对话
                # --- END: 添加 JSON 提取和解析逻辑 ---
            else:
                print("Agent: 未知响应，可能需要更多交互。")
                break
                
        except Exception as e:
            print(f"调用 DeepSeek API 时发生错误: {e}")
            break
    
    print("\n⚠️ Agent 达到最大迭代次数或未能生成最终文案。请检查Prompt或增加迭代次数。")
    return "未能成功生成文案。"

In [37]:
# 测试案例 1: 黑松露抗老面霜
product_name_1 = "黑松露抗老面霜"
tone_style_1 = "活泼甜美"
result_1 = generate_rednote(product_name_1, tone_style_1)

print("\n--- 生成的文案 1 ---")
print(result_1)


🚀 启动小红书文案生成助手，产品：黑松露抗老面霜，风格：活泼甜美

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '黑松露抗老面霜'}
[Tool Call] 查询 milvus 产品数据库：黑松露抗老面霜
Observation: 工具返回结果：2. **黑松露抗老面霜**：运用诺贝尔细胞自噬技术，含黑松露发酵物+胜肽复合物，专为30+熟龄肌抗老设计，夜间厚敷按摩至吸收。
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '黑松露抗老面霜，诺贝尔技术，抗老效果'}
[Tool Call] 模拟生成表情符号，上下文：黑松露抗老面霜，诺贝尔技术，抗老效果
Observation: 工具返回结果：['🤩']
-- Iteration 3 --
[模型生成结果] ```json
{
  "title": "🤩诺贝尔技术加持！黑松露抗老面霜让我一夜回春！",
  "body": "姐妹们！我终于找到了我的抗老救星✨——黑松露抗老面霜！\n\n这款面霜简直是为30+熟龄肌量身定制，核心成分是诺贝尔细胞自噬技术的黑松露发酵物+胜肽复合物，抗老效果直接拉满！\n\n🌟使用体验：\n- 质地丝滑，上脸瞬间吸收，完全不油腻！\n- 夜间厚敷按摩后，第二天皮肤紧致到发光！\n- 细纹淡化效果肉眼可见，连闺蜜都问我是不是偷偷去做了医美！\n\n💖真心推荐给所有抗初老的姐妹，一瓶就能让你告别熬夜脸，重回少女肌！\n\n#抗老面霜 #黑松露护肤 #诺贝尔技术 #熟龄肌救星 #30岁护肤",
  "hashtags": ["#抗老面霜", "#黑松露护肤", "#诺贝尔技术", "#熟龄肌救星", "#30岁护肤"],
  "emojis": ["🤩", "✨", "🌟", "💖", "👏"]
}
```
Agent: 任务完成，成功解析最终JSON文案。

--- 生成的文案 1 ---
{
  "title": "🤩诺贝尔技术加持！黑松露抗老面霜让我一夜回春！",
  "body": "姐妹们！我终于找到了我的抗老救星

In [39]:
# 测试案例 2: 藏红花修护油
product_name_1 = "藏红花修护油"
tone_style_1 = "知性温柔"
result_1 = generate_rednote(product_name_1, tone_style_1)

print("\n--- 生成的文案 2 ---")
print(result_1)


🚀 启动小红书文案生成助手，产品：藏红花修护油，风格：知性温柔

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '藏红花修护油'}
[Tool Call] 查询 milvus 产品数据库：藏红花修护油
Observation: 工具返回结果：15. **藏红花修护油**：运用古法淬炼工艺，含西藏藏红花+摩洛哥坚果油，修复孕纹，每日沐浴后顺时针按摩5分钟。
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '藏红花修护油，修复孕纹，古法淬炼'}
[Tool Call] 模拟生成表情符号，上下文：藏红花修护油，修复孕纹，古法淬炼
Observation: 工具返回结果：['💧']
-- Iteration 3 --
[模型生成结果] ```json
{
  "title": "藏红花修护油｜古法淬炼的孕纹救星✨",
  "body": "最近被这款藏红花修护油彻底圈粉了！💖\n\n作为一个成分党，我特别看重它的配方——西藏藏红花+摩洛哥坚果油，古法淬炼工艺，每一滴都是精华！💧\n\n每天沐浴后，我会取适量在掌心温热，然后以顺时针方向轻轻按摩5分钟，不仅吸收快，还能感受到肌肤被温柔呵护的感觉～坚持用了一周，孕纹明显淡化了！🌟\n\n如果你也在寻找一款温和又高效的修护油，这款真的值得一试！✨",
  "hashtags": ["#藏红花修护油", "#孕纹修复", "#古法淬炼", "#温和护肤", "#成分党"],
  "emojis": ["✨", "💖", "💧", "🌟", "🔥"]
}
```
Agent: 任务完成，成功解析最终JSON文案。

--- 生成的文案 2 ---
{
  "title": "藏红花修护油｜古法淬炼的孕纹救星✨",
  "body": "最近被这款藏红花修护油彻底圈粉了！💖\n\n作为一个成分党，我特别看重它的配方——西藏藏红花+摩洛哥坚果油，古法淬炼工艺，每一滴都是精华！💧\n\n每天沐浴后，我会取适量在掌心温热，然后以顺时针方向轻轻按摩5分钟，不仅吸收快，还能

In [40]:
# 测试案例 3: 钻石微晶眼膜
product_name_1 = "钻石微晶眼膜"
tone_style_1 = "严肃专业"
result_1 = generate_rednote(product_name_1, tone_style_1)

print("\n--- 生成的文案 3 ---")
print(result_1)


🚀 启动小红书文案生成助手，产品：钻石微晶眼膜，风格：严肃专业

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '钻石微晶眼膜'}
[Tool Call] 查询 milvus 产品数据库：钻石微晶眼膜
Observation: 工具返回结果：6. **钻石微晶眼膜**：创新3D剪裁抗引力设计，含六胜肽+钻石粉末，紧致眼周松弛肌肤，贴敷8分钟揭下无需清洗。
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '钻石微晶眼膜 紧致眼周 抗衰老'}
[Tool Call] 模拟生成表情符号，上下文：钻石微晶眼膜 紧致眼周 抗衰老
Observation: 工具返回结果：['💯', '🤩', '💧']
-- Iteration 3 --
[模型生成结果] ```json
{
  "title": "💎钻石微晶眼膜💎 | 8分钟紧致眼周，抗衰老黑科技！",
  "body": "姐妹们！我终于找到了拯救眼周松弛的神器——钻石微晶眼膜！🤩\n\n这款眼膜采用了创新3D剪裁抗引力设计，完美贴合眼周肌肤，内含六胜肽+钻石粉末成分，8分钟就能感受到紧致提拉的效果！💯 揭下后无需清洗，懒人福音啊！\n\n💧使用感受：\n- 贴上去冰冰凉凉，超级舒服！\n- 揭下后眼周明显紧致，细纹淡化效果肉眼可见！\n- 钻石粉末的微晶技术真的绝了，肌肤瞬间提亮！\n\n抗衰老的路上，这款眼膜绝对是你的必备单品！💖",
  "hashtags": ["#钻石微晶眼膜", "#抗衰老", "#紧致眼周", "#六胜肽", "#懒人护肤"],
  "emojis": ["💎", "🤩", "💯", "💧", "💖"]
}
```
Agent: 任务完成，成功解析最终JSON文案。

--- 生成的文案 3 ---
{
  "title": "💎钻石微晶眼膜💎 | 8分钟紧致眼周，抗衰老黑科技！",
  "body": "姐妹们！我终于找到了拯救眼周松弛的神器——钻石微晶眼膜！🤩\n\n这款眼膜采用了创新3D剪裁抗引力设计，完美贴合眼周肌